In [ ]:
import os
import time
import csv
import re
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.pydantic_v1 import BaseModel, Field

# --- 1. 설정 ---
load_dotenv()
API_KEY = os.getenv('GEMINI_API')

if not API_KEY:
    print("오류: GEMINI_API_KEY 환경 변수를 설정해주세요.")
    exit()

LANGCHAIN_MODEL_NAME = 'gemini-2.0-flash'

try:
    llm = ChatGoogleGenerativeAI(
        google_api_key=API_KEY,
        model=LANGCHAIN_MODEL_NAME,
        temperature=0.7,
        top_p=1,
        top_k=1,
    )
except Exception as e:
    print(f"LangChain LLM ({LANGCHAIN_MODEL_NAME}) 초기화 중 오류 발생: {e}")
    print("GEMINI_API_KEY가 정확하고, langchain_google_genai 라이브러리가 올바르게 설치되었는지, 모델명이 유효한지 확인해주세요.")
    exit()

# DC_GALLERY_URL = "https://gall.dcinside.com/mgallery/board/lists/?id=platinalab"
DC_GALLERY_URL = "https://gall.dcinside.com/board/lists?id=baseball_new11"
BASE_DC_URL = "https://gall.dcinside.com" # 링크 생성 시 사용
TARGET_POST_COUNT = 300
OUTPUT_CSV_FILE = "./data/defamatory_posts_db_langchain.csv"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- 2. 웹 스크레이핑 함수 ---

def get_post_links_and_titles(gallery_url, target_count):
    """갤러리 목록 페이지에서 게시글 링크와 제목을 수집합니다."""
    post_details = []
    page = 1
    collected_count = 0
    pbar = tqdm(total=target_count, desc="게시글 링크 수집 중")

    while collected_count < target_count:
        try:
            current_url = f"{gallery_url}&page={page}"
            response = requests.get(current_url, headers=HEADERS, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            # 수정된 선택자: 'table.gall_list' 내부의 'tbody' 내부의 'tr' 태그 중
            # 클래스가 'ub-content'와 'us-post'를 모두 가진 요소 (일반 게시글)
            posts_in_page = soup.select('table.gall_list tbody tr.ub-content.us-post')

            # print(f"Debug (페이지 {page}): 선택자로 찾은 게시글 행 수: {len(posts_in_page)}") # 디버깅용

            if not posts_in_page:
                if page == 1 and collected_count == 0 : # 첫 페이지부터 아무것도 못찾으면 확실히 문제
                     print(f"페이지 {page}에서 게시글을 찾을 수 없습니다. CSS 선택자('table.gall_list tbody tr.ub-content.us-post')가 정확한지, "
                          "또는 웹사이트 구조가 변경되었는지 확인해주세요.")
                else: # 첫 페이지가 아니거나, 이전에 수집한게 있다면 더 이상 없는것으로 간주
                    print(f"페이지 {page}에서 더 이상 게시글을 찾을 수 없어 수집을 중단합니다.")
                break

            for post_row in posts_in_page:
                # 제목 링크 선택자: 게시글 행(post_row) 내부의 'td'태그 중 클래스가 'gall_tit'인 요소 내부의 'a' 태그.
                # 단, 클래스에 'icon_notice' (공지 아이콘)를 포함하는 링크는 제외.
                title_tag = post_row.select_one('td.gall_tit a:not([class*="icon_notice"])')

                if title_tag and title_tag.has_attr('href'):
                    title = title_tag.get_text(strip=True)
                    relative_link = title_tag['href']

                    # 링크가 절대 경로가 아닌 경우 BASE_DC_URL과 결합
                    if relative_link.startswith('/'):
                        link = BASE_DC_URL + relative_link
                    else: # 이미 완전한 URL인 경우 (드묾)
                        link = relative_link
                    
                    # 중복 링크 방지
                    if not any(d['link'] == link for d in post_details):
                        post_details.append({'title': title, 'link': link})
                        collected_count += 1
                        pbar.update(1)

                    if collected_count >= target_count:
                        break
                # else: # 제목 태그를 못 찾을 경우 디버깅
                    # print(f"Debug: tr.ub-content.us-post 내부에서 제목 태그(td.gall_tit a)를 찾지 못했습니다. 행 내용: {str(post_row)[:200]}")

            if collected_count >= target_count:
                break

            page += 1
            # 서버 부하를 줄이기 위해 각 페이지 요청 사이에 약간의 딜레이
            time.sleep(random.uniform(0.5, 1.5)) # 0.5초 ~ 1.5초 사이 랜덤 딜레이

        except requests.exceptions.Timeout:
            print(f"오류: 페이지 {page} 요청 시간 초과. 다음 페이지로 넘어갑니다.")
            page += 1
            time.sleep(1) # 타임아웃 후 잠시 대기
            continue
        except requests.exceptions.RequestException as e:
            print(f"오류: 페이지 {page} 스크레이핑 중 요청 오류 발생 - {e}")
            break # 심각한 오류 시 중단
        except Exception as e:
            print(f"알 수 없는 오류 발생 (페이지 {page} 처리 중): {e}")
            break # 알 수 없는 오류 시 중단
            
    pbar.close()
    if not post_details and target_count > 0:
        print("수집된 게시글이 전혀 없습니다. 프로그램 초기의 CSS 선택자, 네트워크 연결, 또는 대상 웹사이트의 접근성을 확인해주세요.")
    return post_details[:target_count]

def get_post_content(post_url):
    """개별 게시글 페이지에서 본문 내용을 가져옵니다."""
    try:
        response = requests.get(post_url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 본문 내용 선택자 (두 번째 이미지 참고: div.writing_view_box)
        content_div = soup.select_one('div.writing_view_box')
        
        if content_div:
            # 불필요한 태그 제거 (광고, 스크립트, 버튼 등)
            for unwanted_tag in content_div.find_all(['script', 'style', 'iframe', 'ins', 'figure', 
                                                      '.adv_bottom_title', 'div.btn_recommend_box', 
                                                      'div.json_dccon_viewer', 'div.social_area',
                                                      'div.gallery_info_bottom', 'div.related_cont']):
                unwanted_tag.decompose()
            
            text_parts = []
            # div.write_div 내부의 텍스트를 우선적으로 고려하거나, 전체에서 추출
            # 여기서는 writing_view_box 전체에서 추출하는 기존 방식 유지 (이미지 짤방 텍스트 등도 포함될 수 있음)
            for element in content_div.descendants:
                if isinstance(element, str): # NavigableString
                    stripped_text = element.strip()
                    if stripped_text:
                        text_parts.append(stripped_text)
                elif element.name == 'br': # <br> 태그는 줄바꿈으로
                    if text_parts and text_parts[-1] != '\n' and text_parts[-1].strip() != '': # 마지막이 비어있거나 이미 줄바꿈이면 추가 안함
                         text_parts.append('\n')

            content = " ".join(text_parts) # 공백으로 합치고, 연속 공백은 나중에 정리
            content = re.sub(r'\s+', ' ', content).strip() # 연속 공백을 하나로
            content = content.replace(' \n ', '\n').replace('\n ', '\n').replace(' \n', '\n') # 줄바꿈 주변 공백 정리

            max_length = 3000 # Gemini 모델의 컨텍스트 길이를 고려하여 조절
            return content[:max_length]
        else:
            # print(f"Debug: 게시글({post_url})에서 본문 영역('div.writing_view_box')을 찾지 못했습니다.")
            return "본문 내용을 찾을 수 없습니다."
            
    except requests.exceptions.Timeout:
        # print(f"Debug: 게시글({post_url}) 내용 로드 중 타임아웃 발생.")
        return f"게시글 내용 로드 실패 (타임아웃): {post_url}"
    except requests.exceptions.RequestException as e:
        # print(f"Debug: 게시글({post_url}) 내용 로드 중 요청 오류: {e}")
        return f"게시글 내용 로드 실패: {e}"
    except Exception as e:
        # print(f"Debug: 게시글({post_url}) 내용 로드 중 알 수 없는 오류: {e}")
        return f"알 수 없는 오류로 내용 로드 실패: {e}"

# Pydantic 모델 정의 (LLM이 반환할 JSON 구조 명시)
class DefamationAnalysis(BaseModel):
    is_defamatory: bool = Field(description="게시글이 비방글이면 true, 아니면 false")
    reason: str = Field(description="비방글이라고 판단한 경우 그 이유, 아니라면 '해당 없음'")
    confidence_score: float = Field(description="비방글 판단에 대한 신뢰도 점수 (0.0 ~ 1.0), 소수점 둘째자리까지")

def analyze_text_with_langchain(text_content, post_title, post_link):
    """LangChain과 ChatGoogleGenerativeAI를 사용하여 텍스트가 비방글인지 분석합니다."""
    if not text_content or text_content.strip() == "본문 내용을 찾을 수 없습니다." or text_content.startswith("게시글 내용 로드 실패"):
        return {"classification": "분석 불가", "reason": "게시글 내용을 가져올 수 없음", "confidence": 0.0}

    parser = JsonOutputParser(pydantic_object=DefamationAnalysis)
    prompt_template_str = """
    당신은 커뮤니티 게시글을 분석하여, **실질적인 비방 의도**를 가지고 작성되었는지, 특히 **교묘하고 간접적인 방식(비유, 풍자, 은어, 암시 등)으로 특정 대상을 공격하거나 명예를 훼손하려 하는지** 판단하는 고도로 숙련된 언어 분석 전문가입니다. 단순한 감정 표현이나 욕설 사용 자체만으로 비방으로 단정짓지 않고, 전체 맥락 속에서 글의 **숨겨진 의도와 실제 표적**을 파악하는 것이 중요합니다.

    [중요 지침]
    * **욕설과 비방 의도 구분**: 게시물에 욕설이 포함되어 있더라도, 그것이 반드시 비방 목적을 의미하지는 않습니다. 욕설이 사용된 맥락 (예: 단순 강조, 분노 표출, 친밀한 관계에서의 사용 등), 공격 대상의 명확성, 반복성, 전반적인 어조를 종합적으로 고려하여, 단순한 거친 표현과 악의적인 비방 의도를 명확히 구분해야 합니다. 욕설 자체가 아니라, 그 욕설이 **특정 대상을 겨냥하여 명예를 훼손하거나 모욕하려는 의도**로 사용되었는지 여부가 핵심입니다.
    * **맥락 및 대상 중심 판단**: 글의 전체적인 흐름, 사용된 단어의 뉘앙스, 사회적 통념, 그리고 해당 글이 궁극적으로 누구를 향하고 있는지(표적의 유무 및 특정성)를 깊이 있게 분석해주세요. 비판의 대상이 공적인 인물이나 정책일 경우, 그 비판이 공익성을 가지는지, 혹은 악의적인 인신공격에 해당하는지도 고려해주세요.
    * **간접 표현 분석**: 비유, 풍자, 반어법, 이중 의미를 가진 은어, 교묘한 암시 등은 직접적인 표현보다 더 파악하기 어려울 수 있습니다. 이러한 표현들이 특정 대상을 부정적으로 묘사하거나 조롱하려는 의도로 사용되었는지, 문맥적 단서와 사회적 통념을 바탕으로 신중하게 판단해주세요.

    [판단 기준] - 아래 기준 중 하나 이상에 해당하며, 명확한 비방 의도가 감지될 경우 '비방글'로 판단합니다.
    1.  **악의적 직접 공격**: 특정 대상을 명확히 겨냥하여, 의도적으로 명예를 훼손하거나 심각한 모욕감 또는 불쾌감을 유발할 목적으로 욕설, 모욕적인 언어, 인신공격을 사용하는 내용. (단, 대상이 불분명하거나 사회적으로 용인될 수 있는 수준의 단순한 감정 표출, 또는 풍자적 맥락에서의 과장된 표현과는 구분되어야 합니다.)
    2.  **명예훼손성 허위사실/사실 적시**: 개인이나 단체의 사회적 평가를 저해할 수 있는 구체적인 허위 사실을 유포하거나, 진실한 사실이라도 공공의 이익과 무관하게 사생활을 침해하며 비방할 의도가 명확한 내용. (단순 의견, 추측, 가치 판단과는 구분)
    3.  **악의적 조롱/희화화**: 특정 대상을 지속적이고 의도적으로 조롱하거나 우스꽝스럽게 표현하여 사회적 평판을 훼손하고 모멸감을 주려는 목적이 뚜렷한 내용. (단순 유머나 풍자와의 경계를 신중히 판단)
    4.  **혐오 발언**: 성별, 인종, 국적, 출신 지역, 종교, 성적 지향, 장애, 특정 질병, 정치적 또는 사회적 신념 등을 이유로, 특정 집단 또는 개인 구성원에 대해 명백한 적대감, 폭력 선동, 차별을 정당화하거나 조장하며, 이들의 존엄성과 안전을 위협하는 내용.
    5.  **선동/위협**: 특정 개인이나 집단에 대한 구체적인 폭력, 사회적 배제, 사이버 괴롭힘, 신상털이 등을 명시적 또는 암시적으로 선동하거나 위협하여 공포감을 조성하고 명예를 실추시키려는 내용.
    6.  **간접적/우회적 비방**: 표면적으로는 직접적인 공격이 아니지만, 비유(예: 동식물, 사물에 빗댐), 풍자(satire), 반어법, 이중 의미를 가진 은어/줄임말, 교묘한 암시, 질문 형식의 비난 등을 사용하여, 문맥과 사회적 통념을 통해 특정 대상을 비하하거나 명예를 훼손하려는 숨겨진 의도가 명확히 추론되는 내용. (예: 특정 인물/집단을 지칭하는 부정적 은어를 반복 사용, 풍자적 글이나 그림을 통해 교묘히 인신공격)

    [게시글 정보]
    제목: "{post_title}"
    내용:
    ---
    {text_content}
    ---

    [출력 형식] - 반드시 다음 JSON 스키마를 따라주세요.
    {format_instructions}

    [분석 결과 설명 작성 가이드]
    * 비방글로 판단될 경우, 그 이유를 위의 [판단 기준] 번호와 함께, 어떤 표현/맥락 때문에 그렇게 판단했는지 핵심 내용을 간략히 설명해주세요. (예: "1. OOO에 대해 'XXX' 등 모욕적인 욕설을 반복 사용하며 인신공격함.", "6. 특정 정치인을 동물에 비유하며 조롱하고, 정책 실패를 암시적으로 비난함.", "4. XX지역 사람들에 대한 근거 없는 편견을 드러내며 혐오감을 조성함.")
    * 비방글이 아니라면, 이유는 '해당 없음', '단순 의견/감정 표현', '비방 의도가 낮은 풍자적 표현' 등으로 간략히 기술하고, 왜 그렇게 판단했는지 짧게 부연할 수 있습니다. (예: "단순 분노 표출로, 특정 대상을 향한 지속적 비방 의도는 낮음.")
    * 신뢰도 점수는 0.0에서 1.0 사이로 표현하며, 판단에 대한 확신 정도를 나타냅니다. (예: 0.95)
    """
    
    prompt = ChatPromptTemplate.from_template(
        template=prompt_template_str,
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    chain = prompt | llm | parser

    try:
        analysis_result = chain.invoke({
            "post_title": post_title,
            "text_content": text_content
        })
        
        classification = "비방글" if analysis_result.get("is_defamatory") else "일반글"
        reason = analysis_result.get("reason", "이유 분석 실패")
        confidence = analysis_result.get("confidence_score", 0.0)
            
        return {"classification": classification, "reason": reason, "confidence": confidence}

    except Exception as e:
        # print(f"LangChain 분석 중 오류 ({post_link}): {e}") # 상세 오류 확인용
        if "API key not valid" in str(e):
             print("오류: Gemini API 키가 유효하지 않습니다. .env 파일을 확인해주세요.")
        elif "429" in str(e) or "Resource has been exhausted" in str(e) or "quota" in str(e).lower():
             print(f"Gemini API 할당량 초과 가능성 ({post_link}). 잠시 후 수동 재시도 또는 스크립트 재실행 필요.")
             return {"classification": "분석 실패 (API 할당량)", "reason": str(e), "confidence": 0.0} # 재시도 필요 표시
        # 모델이 JSON 형식으로 응답하지 않는 경우도 여기에 포함될 수 있음
        # print(f"Gemini API 응답 처리 중 문제 발생 가능성 ({post_link}). 응답 텍스트: {getattr(e, 'response', '')}")
        return {"classification": "분석 실패", "reason": f"Gemini 응답 처리 오류: {str(e)[:100]}", "confidence": 0.0}


# --- 4. 메인 로직 ---
import random # time.sleep()에 사용
print(f"DCInside 게시글 분석을 시작합니다 (LangChain, 모델: {LANGCHAIN_MODEL_NAME})...")

print(f"'{DC_GALLERY_URL}'에서 최대 {TARGET_POST_COUNT}개의 게시글 링크를 수집합니다.")
posts_to_analyze = get_post_links_and_titles(DC_GALLERY_URL, TARGET_POST_COUNT)

if not posts_to_analyze:
    # get_post_links_and_titles 함수 내에서 이미 상세한 원인 메시지를 출력했을 것이므로, 여기서는 간단히 종료 알림.
    print("프로그램을 종료합니다.")
    return

print(f"\n총 {len(posts_to_analyze)}개의 게시글 링크 수집 완료.")
all_results = []

print("\n게시글 내용 스크레이핑 및 LangChain Gemini API 분석 중...")
for post_detail in tqdm(posts_to_analyze, desc="게시글 분석 중"):
    post_title = post_detail['title']
    post_link = post_detail['link']
    
    post_content = get_post_content(post_link)

    if not post_content or post_content.strip() == "본문 내용을 찾을 수 없습니다." or post_content.startswith("게시글 내용 로드 실패"):
        analysis = {"classification": "분석 불가 (내용 없음)", "reason": "본문 내용을 가져올 수 없습니다.", "confidence": 0.0}
    else:
        analysis_attempts = 0
        max_analysis_attempts = 2 # API 호출 재시도 횟수
        analysis = None
        while analysis_attempts < max_analysis_attempts:
            analysis = analyze_text_with_langchain(post_content, post_title, post_link)
            if analysis and analysis["classification"] != "분석 실패 (API 할당량)":
                break # 할당량 문제가 아니거나 성공이면 종료
            analysis_attempts +=1
            if analysis_attempts < max_analysis_attempts and analysis and analysis["classification"] == "분석 실패 (API 할당량)":
                wait_time = 30 * analysis_attempts # 재시도 시 대기 시간 증가 (30초, 60초)
                print(f"Gemini API 할당량 문제로 {wait_time}초 대기 후 재시도 ({analysis_attempts}/{max_analysis_attempts})...")
                time.sleep(wait_time)
            elif analysis_attempts < max_analysis_attempts : # 그 외 분석 실패 시 짧은 대기 후 재시도
                time.sleep(3)


    all_results.append({
        "title": post_title,
        "link": post_link,
        "content_excerpt": post_content[:200].replace('\n', ' ') + "..." if post_content and isinstance(post_content, str) else "내용 없음",
        "gemini_classification": analysis.get("classification", "분석 실패"),
        "gemini_reason": analysis.get("reason", "N/A"),
        "gemini_confidence": f"{analysis.get('confidence', 0.0):.2f}"
    })
    # 각 게시물 처리 후 짧은 휴식 (서버 및 API에 대한 부하 감소)
    time.sleep(random.uniform(1.2, 2.0)) # 0.8초 ~ 2.0초 사이 랜덤 딜레이

# --- 5. 결과를 CSV 파일로 저장 ---
print(f"\n분석 결과를 '{OUTPUT_CSV_FILE}' 파일로 저장합니다.")
defamatory_posts_count = 0
if not all_results:
    print("저장할 분석 결과가 없습니다.")
else:
    try:
        with open(OUTPUT_CSV_FILE, mode='w', newline='', encoding='utf-8-sig') as file:
            writer = csv.DictWriter(file, fieldnames=["title", "link", "content_excerpt", "gemini_classification", "gemini_reason", "gemini_confidence"])
            writer.writeheader()
            for result in all_results:
                writer.writerow(result)
                if result["gemini_classification"] == "비방글":
                    defamatory_posts_count +=1
        print(f"'{OUTPUT_CSV_FILE}' 파일 저장 완료. 총 {len(all_results)}개 게시글 분석, {defamatory_posts_count}개 비방 의심글 발견.")
    except IOError as e:
        print(f"오류: '{OUTPUT_CSV_FILE}' 파일 저장에 실패했습니다. - {e}")

print("\n프로그램이 종료되었습니다.")

DCInside 게시글 분석을 시작합니다 (LangChain, 모델: gemini-2.0-flash)...
'https://gall.dcinside.com/board/lists?id=baseball_new11'에서 최대 300개의 게시글 링크를 수집합니다.


게시글 링크 수집 중: 100%|██████████| 300/300 [00:06<00:00, 43.93it/s]



총 300개의 게시글 링크 수집 완료.

게시글 내용 스크레이핑 및 LangChain Gemini API 분석 중...


게시글 분석 중:   5%|▌         | 16/300 [00:48<15:16,  3.23s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 50
}
].


Gemini API 할당량 초과 가능성 (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438494&page=1). 잠시 후 수동 재시도 또는 스크립트 재실행 필요.
Gemini API 할당량 문제로 30초 대기 후 재시도 (1/2)...


...

Gemini API 할당량 초과 가능성 (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438254&page=5). 잠시 후 수동 재시도 또는 스크립트 재실행 필요.
Gemini API 할당량 문제로 30초 대기 후 재시도 (1/2)...


게시글 분석 중:  90%|█████████ | 270/300 [18:00<01:32,  3.10s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 38
}
].
게시글 분석 중:  96%|█████████▌| 288/300 [18:56<00:30,  2.58s/it]Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry 

Gemini API 할당량 초과 가능성 (https://gall.dcinside.com/board/view/?id=baseball_new11&no=18438205&page=6). 잠시 후 수동 재시도 또는 스크립트 재실행 필요.
Gemini API 할당량 문제로 30초 대기 후 재시도 (1/2)...


게시글 분석 중: 100%|██████████| 300/300 [20:06<00:00,  4.02s/it]


분석 결과를 './data/defamatory_posts_db_langchain.csv' 파일로 저장합니다.
'./data/defamatory_posts_db_langchain.csv' 파일 저장 완료. 총 300개 게시글 분석, 120개 비방 의심글 발견.

프로그램이 종료되었습니다.
